# Scraping 3D Juegos

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from newspaper import Source
from newspaper import fulltext

## Scraping Articles Site 

In [188]:
review_url = 'https://www.3djuegos.com/novedades/analisis/juegos-ps4/0f37f0f0/fecha/'
html = requests.get(review_url).content
html[:50]

b'<!DOCTYPE html><html lang="es"><head>\n<!--[if lt I'

In [190]:
soup = BeautifulSoup(html, 'lxml')
articles = soup.find_all('h2')

In [24]:
#Extraer títulos

titles = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        titles.append(a.find('a')['title'])
    
titles[0:5]

['FIFA 21 - Análisis',
 'Genshin Impact - Análisis',
 "Crash Bandicoot 4: It's About Time - Análisis",
 'Star Wars: Squadrons - Análisis',
 'Kirby Fighters 2 - Análisis']

In [16]:
#Extraer links

links = [a.find('a')['href'] for a in articles]
    
links[0:4]

['https://www.3djuegos.com/juegos/analisis/37068/0/fifa-21/',
 'https://www.3djuegos.com/juegos/analisis/36571/0/genshin-impact/',
 'https://www.3djuegos.com/juegos/analisis/37086/0/crash-bandicoot-4-its-about-time/',
 'https://www.3djuegos.com/juegos/analisis/36317/0/star-wars-squadrons/']

## Page parsing and link retrieving function for 3D Juegos

In [30]:
def tdjuegos_link_retrieve(num_pages):
    links = []
    titles = []
    
    # pages parser
    for i in range(num_pages):
        url = f"https://www.3djuegos.com/novedades/analisis/juegos/{i+1}pf0f0f0/fecha/"

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find_all('h2')

        # links retrieve
        for a in articles:
            links.append(a.find('a')['href']) 
            titles.append(a.find('a')['title'])
    
    #Deleting non review links
#    for link in links:
#        if 'analisis' not in link:
#            links.remove(link)
                
    return links, titles


In [33]:
num_pages = 100

links, titles = tdjuegos_link_retrieve(num_pages)

In [166]:
links[2994]

'https://www.3djuegos.com/juegos/analisis/11738/0/hard-reset/'

## Scraping Single Review

In [138]:
review_url = 'https://www.3djuegos.com/juegos/analisis/37068/0/fifa-21/'
review_html = requests.get(review_url).content
review_html[:50]

b"<!DOCTYPE html>\n<html lang='es'>\n<head>\n<meta prop"

In [139]:
soup = BeautifulSoup(review_html, 'lxml')

In [140]:
p_tags = soup.find('div', {'class': 'lh27 url_lineas article_body0 mar_temp_0'}).find_all('p')

In [141]:
review = [tag.text for tag in p_tags]
review = ' '.join(review)

In [143]:
review[0:333]

'FIFA 21 no será la entrega más recordada de la saga de simulación futbolística de EA Sports. Ligeras modificaciones jugables, técnicas y modalidades subrayan un pensamiento generalizado de toda la comunidad: hay que dar un buen repaso a la franquicia en la nueva generación... y quitar a Manolo Lama y Paco González en la versión esp'

In [144]:
author = soup.find('a', {'class': 'c7 n'}).text
author

'Toni Piedrabuena'

In [82]:
# Author
'''def tdj_author():
    author = 'None'
    
    try:
        author = soup.find('p', {'class': 'art-aut-wr'}).find('a').text.strip('\n')

    except AttributeError:
        pass
    
    try:
        author = soup.find('li', {'class': 'art-aut-wr'}).text.strip('\n')

    except AttributeError:
        pass

    return author'''

In [173]:
# No Genre, No Company and Platform

def tdjuegos_platform(soup):
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c1o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
   
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c2o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    
    except AttributeError:
        pass

    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c3o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c4o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c5o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
 
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c6o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c34o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass

    finally:
        
        platform = [plat.contents[0] for plat in platform_soup[2:]]        
        platforms = ' '.join(platform)
            
    return platforms

In [174]:
tdjuegos_platform(soup)

'PC PS4 PS5 XSeries XOne Switch'

In [116]:
#Score

score = soup.find('div', {'class': 'nota_ana_3 fftext b nota_interior2'}).text
score = score.replace(',','.')
score = float(score)

6.0

## Create columns

In [175]:

def tdjuegos_dict(links, titles):
    reviews_dict = {}
    i = 0

    for link, title in zip(links, titles):
        try:
            
            #Request content and wait
            review_html = requests.get(link).content

            #Avoid get banned and timeout
            time.sleep(1)

            #Make a Soup and a Newspacer build
            soup = BeautifulSoup(review_html, 'lxml')

            #Author - From Scraping
            author = soup.find('a', {'class': 'c7 n'}).text

            #Company, Genre & Platform - From Scraping
            genre = 'None'
            company = 'None'
            platform = 'None'

            platform = tdjuegos_platform(soup)

            #Text & Cleaning - From Scraping
            p_tags = p_tags = soup.find('div', {'class': 'lh27 url_lineas article_body0 mar_temp_0'}).find_all('p')
            review = [tag.text for tag in p_tags]
            review = ' '.join(review)

            #Score & Clean & Transform - From Scraping
            score = 7

            try:
                score = soup.find('div', {'class': 'nota_ana_3 fftext b nota_interior2'}).text
                score = score.replace(',','.')
                
            except AttributeError:
                pass
            
            score = float(score)
            
            #Add to a dict
            reviews_dict[i] = {'site': '3D Juegos',
                               'url_link': link,
                               'author': author,
                               'game': title,
                               'company': company,
                               'genre': genre,
                               'platform': platform,
                               'text': review,
                               'score': score}
        
        except AttributeError:
            print('error', i)
            pass
        
        i = i+1
        if i % 25 == 0:
            print(i, ': ', link)

    return reviews_dict

In [184]:
result_tdjuegos = tdjuegos_dict(links[1600:2994], titles[1600:2994])

25 :  https://www.3djuegos.com/juegos/analisis/15798/0/a-bird-story/
50 :  https://www.3djuegos.com/juegos/analisis/20648/0/valkyria-chronicles/
75 :  https://www.3djuegos.com/juegos/analisis/16331/0/freedom-wars/
100 :  https://www.3djuegos.com/juegos/analisis/16121/0/the-evil-within/
125 :  https://www.3djuegos.com/juegos/analisis/18681/0/fifa-15/
150 :  https://www.3djuegos.com/juegos/analisis/18509/0/danganronpa-2-goodbye-despair/
175 :  https://www.3djuegos.com/juegos/analisis/18887/0/the-last-of-us-remasterizado/
200 :  https://www.3djuegos.com/juegos/analisis/15828/0/sniper-elite-3/
225 :  https://www.3djuegos.com/juegos/analisis/15711/0/watch-dogs/
250 :  https://www.3djuegos.com/juegos/analisis/17742/0/the-amazing-spider-man-2/
275 :  https://www.3djuegos.com/juegos/analisis/13038/0/chaos-code/
300 :  https://www.3djuegos.com/juegos/analisis/18521/0/steamworld-dig/
325 :  https://www.3djuegos.com/juegos/analisis/17023/0/hakuoki-memories-of-the-shinsengumi/
350 :  https://www.3

### Create DataFrame

In [185]:
tdjuegos = pd.DataFrame.from_dict(result_tdjuegos, orient='index')

tdjuegos

,site,url_link,author,game,company,genre,platform,text,score
0,3D Juegos,https://www.3djuegos.com/juegos/analisis/20431...,Alberto Pastor,Final Fantasy XIII-2 - Análisis,None,None,PC PS3 X360,Con mejores sensaciones que las que nos dejó i...,8.0
1,3D Juegos,https://www.3djuegos.com/juegos/analisis/21193...,Sergio Martín,Grand Theft Auto: Chinatown Wars - Análisis,None,None,PSP DS Android iOS,"Tras su estreno (no demasiado exitoso, por cie...",8.5
2,3D Juegos,https://www.3djuegos.com/juegos/analisis/20682...,Sergio Martín,La Tierra-Media: Sombras de Mordor - El Señor ...,None,None,· PC PS4 XOne,Ya está preparado el primer DLC para el mega h...,7.0
3,3D Juegos,https://www.3djuegos.com/juegos/analisis/10831...,Alejandro Pascual,Caller's Bane - Análisis,None,None,PC Mac Android iOS,Con un sistema de tablero muy vistoso y centrá...,7.5
4,3D Juegos,https://www.3djuegos.com/juegos/analisis/15998...,Alberto Pastor,Lords of Xulima - Análisis,None,None,PC Mac Linux,Explora el rico universo de fantasía de Xulima...,8.0
...,...,...,...,...,...,...,...,...,...
1389,3D Juegos,https://www.3djuegos.com/juegos/analisis/11830...,Sergio Martín,Resident Evil 4 HD - Análisis,None,None,· PS3 X360,"En el año 2005, los usuarios de GameCube (y me...",8.0
1390,3D Juegos,https://www.3djuegos.com/juegos/analisis/8996/...,Álvaro Castellano,From Dust - Análisis,None,None,PC PS3 X360,El nuevo proyecto de Eric Chahi toma forma en ...,8.5
1391,3D Juegos,https://www.3djuegos.com/juegos/analisis/8386/...,Álvaro Castellano,Gears of War 3 - Análisis,None,None,X360,El final ha llegado. La trilogía de Marcus Fen...,9.5
1392,3D Juegos,https://www.3djuegos.com/juegos/analisis/10489...,Álvaro Castellano,Dead Space - Análisis,None,None,PC PS3 X360 Android iOS,La genial saga de acción y terror Dead Space a...,8.4


In [186]:
tdjuegos.to_csv('../data/tdjuegos_1400_3000l.csv', index=False)

In [198]:
tdjuegos.iloc[::-1].head(50)

,site,url_link,author,game,company,genre,platform,text,score
1393,3D Juegos,https://www.3djuegos.com/juegos/analisis/12780...,Álvaro Castellano,Real Racing 2 - Análisis,None,None,Android iOS,La mejor simulación de velocidad regresa a los...,8.7
1392,3D Juegos,https://www.3djuegos.com/juegos/analisis/10489...,Álvaro Castellano,Dead Space - Análisis,None,None,PC PS3 X360 Android iOS,La genial saga de acción y terror Dead Space a...,8.4
1391,3D Juegos,https://www.3djuegos.com/juegos/analisis/8386/...,Álvaro Castellano,Gears of War 3 - Análisis,None,None,X360,El final ha llegado. La trilogía de Marcus Fen...,9.5
1390,3D Juegos,https://www.3djuegos.com/juegos/analisis/8996/...,Álvaro Castellano,From Dust - Análisis,None,None,PC PS3 X360,El nuevo proyecto de Eric Chahi toma forma en ...,8.5
1389,3D Juegos,https://www.3djuegos.com/juegos/analisis/11830...,Sergio Martín,Resident Evil 4 HD - Análisis,None,None,· PS3 X360,"En el año 2005, los usuarios de GameCube (y me...",8.0
1388,3D Juegos,https://www.3djuegos.com/juegos/analisis/11829...,Sergio Martín,Resident Evil 4 HD - Análisis,None,None,· PS3 X360,"En el año 2005, los usuarios de GameCube (y me...",8.0
1387,3D Juegos,https://www.3djuegos.com/juegos/analisis/10996...,Alberto Pastor,Renegade Ops - Análisis,None,None,PC PS3 X360,"Un villano muy malo, explosiones por doquier y...",8.6
1386,3D Juegos,https://www.3djuegos.com/juegos/analisis/10997...,Alberto Pastor,Renegade Ops - Análisis,None,None,PC PS3 X360,"Un villano muy malo, explosiones por doquier y...",8.6
1385,3D Juegos,https://www.3djuegos.com/juegos/analisis/6639/...,Alberto Pastor,Trackmania 2: Canyon - Análisis,None,None,PC,"Videojuegos de velocidad los hay a montones, p...",8.8
1384,3D Juegos,https://www.3djuegos.com/juegos/analisis/9591/...,Álvaro Castellano,Ico and Shadow of the Colossus: The Collection...,None,None,· PS3,Los dos videojuegos que pusieron en el mapa al...,9.2


In [183]:
display(links[1599])

'https://www.3djuegos.com/juegos/analisis/19630/0/captain-toad-treasure-tracker/'